# Дисклеймер
Эту тетрадку нужно запускать в колабе или в vast.ai. Не мучатесь с установкой библиотек и с обучением на cpu.

In [ ]:
# !apt-get install unzip

In [1]:
%pip install tokenizers matplotlib scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# в vast ai или в последних версия jupyter может не работать автозаполнение, установка этой либы и перезагрука кернела помогает
# %pip install --upgrade jedi==0.17.2

Большая часть этого семинара пересекается с видео от Andrey Karpathy - https://www.youtube.com/watch?v=kCc8FmEb1nY Возможно его объяснения покажутся вам лучше. 
Его видео более детальное и объемное, но фокус в нем скорее на GPT (декодере) для простой генерации текста, и еще оно на torch. 

# Транформеры для машинного перевода

На huggingface очень много предобученных трансформеров, которые позволят вам решить очень большой процент рабочих/исследовательских задач. Однако бывают ситуации, когда нужной предобученной модели нет или она работает не очень хорошо. Обучить очень большую модель вряд ли получится (нужны видеокарты), но вот для средних моделей и специфичных задач может хватить даже колаба/каггла.

Давайте обучим свой небольшой трансформер на задаче машинного перевода. Это самая классическая seq2seq задача и подход, который мы размерем, применим и для других аналогичных задач. Для того, чтобы обучить модель решать какую-то другую seq2seq задачу вам нужно будет только подставить другие данные. Под seq2seq вообще при желании можно подвести любую задачу (классификация - это предсказание последовательности длинной 1 или можно предсказыть названия классов текстом). 

In [2]:
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [3]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2023-03-19 10:17:37--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115,72M  36,9MB/s    in 3,1s    

2023-03-19 10:17:40 (36,9 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2023-03-19 10:17:40--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64,62M  35,3MB/s    in 1,8s    

2023-03-19 10:17:42 (35,3 MB/s) - ‘op

In [4]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [5]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

Пример перевода с английского на русский

In [6]:
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса. Будем использовать wordpiece токенайзер из библиотеки tokenizers от huggingface. Wordpiece разбивает текст на токены, которые могут быть как целыми словами, так и кусками слов и даже отдельными символами. 
В токенизатор мы передаем еще несколько специальных символов - UNK (это символ замена для неизвестных слов), START - этот токен будет добавлен в начало текста, END - этот символ будет добавлен в конце, PAD - это индекс паддинга

В токенезаторе для целевого языка по идее нам не нужен UNK, потому что в переводном тексте технических токенов быть не должно, но мы не можем без него обойтись, потому что мы ограничиваем размер словаря. Можно было бы построить словарь так, чтобы в нем были все возможные токены и тогда UNK был бы не нужен, но тогда размер словаря будет слишком большим. Но мы всегда можем отфильтровать UNK из перевода, просто не разрешая модели предсказывать UNK.

In [7]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [8]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

Когда мы будет раскодировать предсказанные индексы в слова нам пригодится декодер. Лучше использовать готовый из той же библиотеки. Он сам склеит символьные нграммы в слова. Без него нам нужно было бы писать правила, чтобы избавиться от вот такого

In [9]:
# без декодера
' '.join(tokenizer_ru.encode('Пример текста с редким словом').tokens)

'пример текста с ред ##ким словом'

In [10]:
# c готовым декодером
tokenizer_ru.decoder.decode(tokenizer_ru.encode('Пример текста с редким словом').tokens)

'пример текста с редким словом'

### ВАЖНО!

Токенизатор - это неотъемлимая часть модели, поэтому не забывайте сохранять токенизатор вместе с моделью. Если вы забудете про это и переобучите токенизатор, то индексы токенов разойдутся и веса модели будут бесполезны. 

In [11]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [12]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

##### Переводим текст в индексы. 

Для исходного языка нам не нужно никаких дополнительных токенов, поэтому просто вызываем токенизатор.   
А вот для целевого в начало нужно добавить токен '[START]', а в конец '[END]'.  
Эти теги понадобятся, когда мы будем генерировать тексты с уже обученной моделью - тэг [START] мы будем подавать в самом начале перевода, чтобы состояние декодера не было пустым (ниже вы увидите, почему оно не может быть пустым), а [END] будем использовать, чтобы понимать когда остановить перевод (если модель предсказала [END] - значит перевод окончен)

In [15]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids 


Кодируем и паддим

In [16]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, True) for t in ru_sents]

Как обычно мы должны выбрать максимальную допустимую длину. Максимальные значения слишком большие

In [17]:
max_len_en = np.max([len(x) for x in X_en])
max_len_ru = np.max([len(x) for x in X_ru])

In [18]:
max_len_en, max_len_ru

(17863, 19389)

Если посмотреть на перцентили, то будет видно, что это скорее всего выбросы. Большая часть текстов короткие.

In [36]:
print('Исходный язык')
print('50 % текстов <= ', np.percentile([len(x) for x in X_en], 50))
print('75 % текстов <= ', np.percentile([len(x) for x in X_en], 75))
print('90 % текстов <= ', np.percentile([len(x) for x in X_en], 90))
print('95 % текстов <= ', np.percentile([len(x) for x in X_en], 95))
print('99 % текстов <= ', np.percentile([len(x) for x in X_en], 99))

Исходный язык
50 % текстов <=  10.0
75 % текстов <=  18.0
90 % текстов <=  33.0
95 % текстов <=  45.0
99 % текстов <=  75.0


In [35]:
print('Целевой язык')
print('50 % текстов <= ', np.percentile([len(x) for x in X_ru], 50))
print('75 % текстов <= ', np.percentile([len(x) for x in X_ru], 75))
print('90 % текстов <= ', np.percentile([len(x) for x in X_ru], 90))
print('95 % текстов <= ', np.percentile([len(x) for x in X_ru], 95))
print('99 % текстов <= ', np.percentile([len(x) for x in X_ru], 99))

Целевой язык
50 % текстов <=  11.0
75 % текстов <=  20.0
90 % текстов <=  36.0
95 % текстов <=  48.0
99 % текстов <=  79.0


Давайте для каждого из языков возьмем значения, которые покрывают 95 % всех данных

In [39]:
# обратите внимание, что в seq2seq длины могут быть разными
max_len_en, max_len_ru = 45, 48

По умолчанию паддинг делается нулями, но нам это не подходит, так как в постороенных словарях 0 - это тэг UNK. Вытащим тэг паддинга из словаря и будет использовать его

In [40]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

1

In [41]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_ru_out = tf.keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post', 
              value=PAD_IDX)

X_ru_dec = tf.keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1, 
              padding='post', value=PAD_IDX)

На вход модели мы будем подавать три матрицы.

Первая - X_en - это индексы токенов в английских текстах (тут все как и в предыдущих семинарах)

In [23]:
en_sents[0]

"yeah, that's not exactly..."

In [24]:
X_en[0]

array([3280,   13, 2763,    8,   58, 2808, 5148, 2856,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1], dtype=int32)

А вот русскоязычные тексты мы будем подавать через две матрицы - одна будет использоваться как вход для декодера (X_ru_dec), а вторая как правильные ответы (X_ru_out). 

Если присмотреться, то можно заметить что X_ru_dec и X_ru_out отличаются на 1 шаг (X_ru_out это X_ru_dec сдвинутый вправо)

In [25]:
ru_sents[0]

'да, но не совсем...'

In [26]:
X_ru_dec[0]

array([   2, 2588,   15, 2589, 2513, 5362, 2643,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32)

In [27]:
X_ru_out[0]

array([2588,   15, 2589, 2513, 5362, 2643,    3,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32)

Такая структура внутри модели будет использована, чтобы предсказывать следующее слово по текущему контексту. Грубо говоря, модель будет интерпретировать эти две последовательности индексов, как пары контекст-продолжение. 

Вот это:  
2, 2588, 15, 2589, 2513, 5362, 2643   
2588, 15, 2589, 2513, 5362, 2643,    3

Превратится для модели в:  
2 - 2588  
2, 2588 - 15  
2, 2588, 15, 2589 - 2513  
2, 2588, 15, 2589, 2513 - 5362  
2, 2588, 15, 2589, 2513, 5362 - 2643  
2, 2588, 15, 2589, 2513, 5362, 2643 - 3  

(как это будет сделано показано дальше в разделе про маски)

In [28]:
# миллион примеров 
X_en.shape, X_ru_out.shape

((1000000, 30), (1000000, 31))

Разделяем на трейн и тест

In [29]:
(X_en_train, X_en_valid, 
X_ru_dec_train, X_ru_dec_valid, 
X_ru_out_train, X_ru_out_valid) = train_test_split(X_en, 
                                                  X_ru_dec, 
                                                  X_ru_out, 
                                                  test_size=0.05)

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://www.tensorflow.org/text/tutorials/transformer

## Имплементация трансформера

Главное в трансформере это механизм внимания. Функция scaled_dot_product_attention преобразует query, key, value вектора в обновленные вектора с помощью механизма внимания.

Визуализация этих шагов (источник: https://jalammar.github.io/illustrated-transformer/):
![](https://jalammar.github.io/images/t/self-attention-output.png "")

Пройдитесь по каждой строчке в этой функции и сопоставьте с визуализацией.

In [51]:
def scaled_dot_product_attention(query, key, value, mask):
    # Считаем скалярное произведение между запросом (query) и ключом (key), транспонируя ключ
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # Получаем глубину (размерность) ключа и преобразуем ее во float
    depth = tf.cast(tf.shape(key)[-1], tf.float32)

    # Делим результат скалярного произведения на квадратный корень из глубины
    # Это делается для уменьшения влияния больших значений и стабилизации градиентов во время обучения
    logits = matmul_qk / tf.math.sqrt(depth)

    # Если есть маска, применяем ее к логитам, чтобы обнулить нежелательные значения
    if mask is not None:
        logits += (mask * -1e9)

    # Применяем функцию softmax для получения весов внимания
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # Умножаем веса внимания на значения (value) для получения итогового результата
    output = tf.matmul(attention_weights, value)

    return output

В этой функции query, key и value вектора подаются на вход, но как они получаются из эмбедингов?

Это реализовано в классе MultiHeadAttention, который уже является самостоятельным слоем, на котором потом будет построен блок трансформера как в энкодере так и в декодере.

В этом слое можно увидеть три отдельных полносвязных слоя одинаковой размерности (d_model), которые применяются к inputs, чтобы получить q, k, v вектора.  
Можно заметить, что в функции call, разделение на q,k,v происходит еще раньше - они просто достаются из inputs по ключу, как из словаря. Но если немного промотать вперед до encoder_layer , то можно убедиться, что при вызове MultiheadAttention в эти переменные подставляются одно и то же (нажмите CTRL+F и введите #call_mha, чтобы перейти к этому моменту и потом вернутся сюда). 

А если сделать еще один шаг до encoder, то можно увидеть, что такое inputs - это эмбединги слов (CTRL+F и введите #inputs). 



In [43]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads  # количество голов для внимания
        self.d_model = d_model  # размерность вектора модели

        # Убеждаемся, что размерность модели делится нацело на количество голов
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads  # размерность каждой головы

        # Создаем полносвязные слои для запроса, ключа и значения
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        # Создаем последний полносвязный слой
        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        # Разделяем входные данные на головы
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Пропускаем запрос, ключ и значение через соответствующие полносвязные слои
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Разделяем запрос, ключ и значение на головы 
        # то есть просто разрезаем вектора на num_heads частей 
        # и сравниваем все части между собой
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Выполняем механизм внимания с масштабированным скалярным произведением
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # Объединяем головы вместе (склеиваем векторы в один)
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # Пропускаем объединенное внимание через последний полносвязный слой
        outputs = self.dense(concat_attention)

        return outputs


#### Теперь наверное самый сложный момент - почему это MultiHead, а не просто Attention layer? 

В MultiheadAttention используются не по одному q,k,v вектору, а сразу по N векторов, но меньшего размера. В классе выше N задается через парметр num_heads и размерность отдельного q,k или v вектора будет равна d_model/num_heads. 

Внимание рассчитывается между соответствующими парами q и v внутри каждой головы, а итоговый вектор получается объединением результата внимания от всех голов (его размерность будет равна d_model).  

В статье illustrated transformer этот момент визуализируется вот так:
![](https://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png)

Но реализация в TF немного отличается. На этой картинке на шаге 3 показано, что полносвязные слои, через которые получаются q,k,v векторы у каждой головы свои - и их количество не 3, а 3 умножить на количество голов. А вот в TF полносвязных слоя всего 3, но дальше вектора просто разрезаются на куски и начаная с шага 4, все уже совпадает. 

Если задуматься, то разницы между этими подходами нет. Полносвязный слой это матрица, на которую умножается входной вектор, в результате чего получается другой вектор. И каждое значение в новом векторе - это результат попарного умножения и суммы значений в исходном векторе и отдельной колонки в матрице. Таким образом, уможить на матрицу и разрезать результат на три куска - это то же самое, что разрезать матрицу на три куска и умножить вектор на каждый из кусков отдельно:

![](https://i.ibb.co/9Yg580J/image.png)


Давайте еще отдельно посмотрим как происходит разбиение на головы и как потом выглядит скалярное произведение для отдельного куска.

Инициализируем класс

In [44]:
mha = MultiHeadAttention(d_model=10, num_heads=2)

Сгенерируем батч из одного текста с двумя словами и размером эбмединга 3

In [47]:
inp = np.random.normal(size=(1, 2, 3)) 

In [48]:
inp

array([[[-0.21667951, -1.84023019, -1.19507623],
        [ 0.99459661, -1.53305878, -1.73494779]]])

Сформируем входной словарь для класса

In [49]:
inputs = {'query': inp,'key': inp, 'value': inp, 'mask':None}

Применим слой к входному словарю

In [52]:
result = mha(inputs)

На выходе получилось, что для каждого слова у нас теперь эмдединг размером 10 (мы указали такой d_model выше)

In [53]:
result.shape

TensorShape([1, 2, 10])

Давайте посмотрим как это вектор вычислялся внутри класса по шагам

In [54]:
# преобразуем эмбединг в q,k,v вектора
q = mha.query_dense(inputs['query'])
k = mha.query_dense(inputs['key'])
v = mha.query_dense(inputs['value'])

In [55]:
# размерность не изменилась так как мы еще не порезали на головы
q.shape

TensorShape([1, 2, 10])

In [56]:
# разрезаем каждый вектор на куски (головы)
qh = mha.split_heads(q, tf.shape(q)[0])
kh = mha.split_heads(k, tf.shape(k)[0])
vh = mha.split_heads(v, tf.shape(v)[0])

В результате получается тензор с размерностями (количество текстов x количество слов x количество голов x размерность вектора головы)

In [78]:
# мы указали num_heads 2 - поэтому размерность каждого куска 5 (10 разделить на два)
qh.shape

TensorShape([1, 2, 2, 5])

In [82]:
tf.transpose(kh).shape

TensorShape([5, 2, 2, 1])

Далее вычисляется скалярное произведение между q и k кусками. На вход подаются тензоры, но тензорфлоу умеет работать с таким форматом тоже

In [58]:
matmul_qk = tf.matmul(qh, kh, transpose_b=True, )

In [59]:
# На выходе получится тензор вот такой размерности
# 5 изменилось на 2
# так как для каждого куска у нас по две близости (до каждого слова включая себя)
matmul_qk.shape

TensorShape([1, 2, 2, 2])

Применим софтмакс, чтобы нагляднее увидеть близости

In [60]:
tf.nn.softmax(matmul_qk, axis=-1).numpy().round(2)

array([[[[0.74, 0.26],
         [0.37, 0.63]],

        [[0.19, 0.81],
         [0.02, 0.98]]]], dtype=float32)

Далее эти близости используются как веса для суммы value кусков и потом результат по каждому кусочку собирается обратно в один вектор размерности (1, 2, 10)

### Маски

Еще один важный момент это то как используются маски, чтобы занулить некоторые близости.
Есть два вида масок - маска для паддинга и маска для слов в будущем. 
Маска для паддинга используется и в энкодере и в декодере, а маска для слов в будущем только для декодера.

In [61]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [62]:
create_padding_mask([[1,2,3]]) # 1 подставилась для индекса который равен PAD_IDX

<tf.Tensor: shape=(1, 1, 1, 3), dtype=float32, numpy=array([[[[1., 0., 0.]]]], dtype=float32)>

Маска для будущего создает треугольную матрицу, которая накладывается на близости в self-attention части декодера. Как раз с помощью этой маски и реализуется разбиение входных последовательностей на контекст-продолжение - сначала вычисляется внимание до всех слов, даже в будущем, но потом полученные близости слов в будущем просто зануляются и не используются.

In [64]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [65]:
# для декодера нам нужно замаскировать следующие токены
# так как мы пытаемся их сгенерировать
# для этого создается вот такая маска
mask = 1 - tf.linalg.band_part(tf.ones((3, 3)), -1, 0)
mask

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 1.],
       [0., 0., 1.],
       [0., 0., 0.]], dtype=float32)>

In [66]:
# в слое с вниманием эта маска будет использоваться чтобы занулить близости с токенами из будущего
(mask * -100)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[  -0., -100., -100.],
       [  -0.,   -0., -100.],
       [  -0.,   -0.,   -0.]], dtype=float32)>

In [67]:
# эта функция также проверяет и паддинг и если он есть то и его тоже замаскирует
create_look_ahead_mask([[1,3,2]])

<tf.Tensor: shape=(1, 1, 3, 3), dtype=float32, numpy=
array([[[[1., 1., 1.],
         [1., 0., 1.],
         [1., 0., 0.]]]], dtype=float32)>

In [69]:
create_look_ahead_mask([[2,4,3]])

<tf.Tensor: shape=(1, 1, 3, 3), dtype=float32, numpy=
array([[[[0., 1., 1.],
         [0., 0., 1.],
         [0., 0., 0.]]]], dtype=float32)>

In [101]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [102]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    #call_mha
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [103]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    #inputs (они тут называются outputs но это просто такой нейминг, 
    # этот параметр передается в encoder_layer первым и encoder_layer будет считать его inputs)
    # outputs он тут называется для удобства, так как он будет перезаписываться
    # чтобы на вход следующему блоку подавать уже не эмбединги,
    # а то что получится как результат предыдущего блока
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [104]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [105]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [106]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [107]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [108]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

Определяем параметры модели. Есть стандартные наборы параметров для обучения маленькой, средней или большой модели. Их можно посмотреть в оригинальной статье про трансформер - https://arxiv.org/pdf/1706.03762.pdf

Мы возьмем маленький, т.к. он требует меньше ресурсов. НО для практической задачи есть смысл использовать модель побольше, есть статьи в которые показано, что большие трансформеры обучать эффективнее (требуется меньше примеров) -  https://arxiv.org/pdf/2002.11794.pdf.

In [109]:
tf.keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8

UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_en, max_len_ru])

#     learning_rate = CustomSchedule(D_MODEL)

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
#         y_true = tf.reshape(y_true, shape=(-1, max_len_ru - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                                monitor='val_loss',
                                                verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Трансформеры - большие модели, обучаться они должны долго. Размер батча здесь уже имеет большое значение: сликом маленький батч приведет к сильному увеличению времени на обучение, но сильно большим его поставить не получится, т.к. модель большая и быстро займет всю видеокарту.

In [115]:
# Обратите внимание на то как данные подаются в модель 
# помимо текста в модель еще нужно передать целевую последовательность
# но не полную а без 1 последнего элемента
# а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
# мы сравниваем выход из модели с целевой последовательностью уже с этим последним элементом

# сдвинутые последовательности создаются выше
# X_ru_dec - это переводной текст без последнего элемента
# X_ru_out - это переводной текст с последним элементом

model.fit((X_en_train, X_ru_dec_train), X_ru_out_train, 
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=500,
             epochs=5,
             callbacks=[checkpoint]
             )
    
    
    

Epoch 1/5
1900/1900 [==============================] - ETA: 0s - loss: 0.9919 - accuracy: 0.2326
Epoch 1: val_loss improved from 1.12355 to 1.11810, saving model to model_ruen
1900/1900 [==============================] - 453s 239ms/step - loss: 0.9919 - accuracy: 0.2326 - val_loss: 1.1181 - val_accuracy: 0.2243
Epoch 2/5
1900/1900 [==============================] - ETA: 0s - loss: 0.9625 - accuracy: 0.2359
Epoch 2: val_loss improved from 1.11810 to 1.11379, saving model to model_ruen
1900/1900 [==============================] - 454s 239ms/step - loss: 0.9625 - accuracy: 0.2359 - val_loss: 1.1138 - val_accuracy: 0.2255
Epoch 3/5
1900/1900 [==============================] - ETA: 0s - loss: 0.9388 - accuracy: 0.2385
Epoch 3: val_loss did not improve from 1.11379
1900/1900 [==============================] - 453s 238ms/step - loss: 0.9388 - accuracy: 0.2385 - val_loss: 1.1146 - val_accuracy: 0.2260
Epoch 4/5
1900/1900 [==============================] - ETA: 0s - loss: 0.9191 - accuracy: 0.2

Можно попробовать перевести рандомное предложение и оценить качество модели самому

In [140]:
def translate(text):
    input_ids = encode(text, tokenizer_en, target=False)

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post', 
                                       # важно не забыть паддинг с нужным id
                                       value=tokenizer_en.token_to_id('[PAD]'))

    
    
    output_ids = [tokenizer_ru.token_to_id('[START]') ]
    
    pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False).numpy()
 
    
    while pred.argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[END]'), 
                                                            ]:
        if len(output_ids) >= max_len_ru:
            break
        # можно занизить скор тэга UNK чтобы он никогда не генерировался
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100

        output_ids.append(pred.argmax(2)[0][-1])
        pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False).numpy()

    return tokenizer_ru.decode(output_ids[1:], )


In [141]:
translate("Transformer")

'трансформатор'

In [142]:
translate("can you translate this sentence?")

'вы можете перевести это предложение?'

In [143]:
translate("please translate this sentence into russian")

'пожалуйста перевести это предложение в русский'

In [144]:
# translate("")

In [145]:
# почему без старт и енд в енкодере хуже???
# ссылки на доп материалы
# как примменяется паддинг маск к атеншену

# BLEU

Оценивание машинного перевода очень сложная задача. Главная сложность в том, что хороших переводов одного текста может быть несколько, а есть обычно только 1. Технического решения этой проблемы пока не придумали и если нужна достоверная оценка, то используют людей или даже профессиональных переводчиков. Для автоматической оценки используется простая метрика BLEU. Она считает пересечения между двумя переводами на уровне отдельных токенов, биграммов, триграммов и четырехграммов. Чем больше полных пересечений, тем лучше. Это конечно очень сильное упрощение, но ничего лучше пока особо не придумали

Реализация bleu есть в nltk

In [146]:
%pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 1.5 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 4.4 MB/s  eta 0:00:01
     |████████████████████████████████| 96 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 772 kB 37.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [147]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


Давайте расчитаем BLEU на 10 примерах из тестового корпуса

In [148]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [149]:
translations = []

for i in range(10):
    translations.append(translate(en_sents_test[i]))

In [153]:
en_sents_test[:10]

['if you only stay there.',
 "i don't know how you do it, pop, carrying these boxes around every day.",
 'we might have a slight edge in mediation.',
 'how long is it going to take you to get him what he needs?',
 "on 1 april president of the nagorno karabagh republic bako sahakyan met head of the general staff of the republic of armenia's armed forces colonel-general yuri khachaturov.",
 'mr priesner also noted that the e-justice management system has not only improved case management, but has also led to a significant streamlining in procedures.',
 "you don't like chicken noodle soup?",
 'posted: 14 may 2005, 20:31',
 'now, for a minute, i thought maybe he was being tailed.',
 '« : 26 октябрь 2017, 06:50:24 »']

In [151]:
ru_sents_test[:10]

['только бы не вылететь.',
 'и как ты только справляешься, папа, таская эти коробки взад-вперед целый день.',
 'возможно, у нас есть небольшое преимущество в переговорах.',
 'сколько времени вы будете делать то, что ему нужно?',
 '1 апреля президент нкр бако саакян принял начальника генштаба вооруженных сил республики армения генерал-полковника юрия хачатурова.',
 'г-н приснер также упомянул, что система электронного правосудия не только позволила улучшить процесс ведения дел, но также способствует значительному упорядочению процедур.',
 '- неплохо, да.',
 'posted: 15 dec 2006, 00:07',
 'и на минутку я подумал, что за ним могут следить.',
 '«: 11 октябрь 2011, 17:15:34»']

BLEU очень сильно зависит от токенизации. В нашем случае это не проблема так как мы оцениваем одну модель. Но если бы мы хотели сравнить несколько, то нужно было бы создавать 1 общий токенизатор и рассчитывать через него

In [152]:
bleus = []

for i, t in enumerate(translations):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True))

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [141]:
(sum(bleus)/len(bleus))*100

0.5141598784548078

BLEU обычно показывают не от 0 до 1, а от 0 до 100. 